In [1]:
import findspark
findspark.init()
from pyspark import SparkContext
from pyspark.sql import SparkSession
spark = SparkSession.builder \
     .appName("SparkSession") \
     .getOrCreate()

sc = spark.sparkContext 
sc

<SparkContext master=yarn appName=SparkSession>

In [2]:
amazon_data = spark.read.json("s3://qianyielva/video")

In [3]:
amazon_data.printSchema()

root
 |-- asin: string (nullable = true)
 |-- helpful: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- overall: double (nullable = true)
 |-- reviewText: string (nullable = true)
 |-- reviewTime: string (nullable = true)
 |-- reviewerID: string (nullable = true)
 |-- reviewerName: string (nullable = true)
 |-- summary: string (nullable = true)
 |-- unixReviewTime: long (nullable = true)



In [4]:
amazon_data.show(10)

+----------+--------+-------+--------------------+-----------+--------------+--------------------+--------------------+--------------+
|      asin| helpful|overall|          reviewText| reviewTime|    reviewerID|        reviewerName|             summary|unixReviewTime|
+----------+--------+-------+--------------------+-----------+--------------+--------------------+--------------------+--------------+
|0700099867| [8, 12]|    1.0|Installing the ga...| 07 9, 2012|A2HD75EMZR8QLN|                 123|Pay to unlock con...|    1341792000|
|0700099867|  [0, 0]|    4.0|If you like rally...|06 30, 2013|A3UR8NLLY1ZHCX|Alejandro Henao "...|     Good rally game|    1372550400|
|0700099867|  [0, 0]|    1.0|1st shipment rece...|06 28, 2014|A1INA0F5CWW3J4|Amazon Shopper "M...|           Wrong key|    1403913600|
|0700099867| [7, 10]|    3.0|I got this versio...|09 14, 2011|A1DLMTOTHQ4AST|            ampgreen|awesome game, if ...|    1315958400|
|0700099867|  [2, 2]|    4.0|I had Dirt 2 on X...|06 14

In [7]:
data = amazon_data.drop("helpful").drop("reviewerID").drop("unixReviewTime").drop("reviewTime").drop("reviewerName")

In [8]:
data.show(10)

+----------+-------+--------------------+--------------------+
|      asin|overall|          reviewText|             summary|
+----------+-------+--------------------+--------------------+
|0700099867|    1.0|Installing the ga...|Pay to unlock con...|
|0700099867|    4.0|If you like rally...|     Good rally game|
|0700099867|    1.0|1st shipment rece...|           Wrong key|
|0700099867|    3.0|I got this versio...|awesome game, if ...|
|0700099867|    4.0|I had Dirt 2 on X...|              DIRT 3|
|0700099867|    4.0|Overall this is a...|Good racing game,...|
|0700099867|    5.0|Loved playing Dir...|A step up from Di...|
|0700099867|    1.0|I can't tell you ...|Crash 3 is correc...|
|0700099867|    4.0|I initially gave ...|A great game ruin...|
|0700099867|    2.0|I still haven't f...|Couldn't get this...|
+----------+-------+--------------------+--------------------+
only showing top 10 rows



In [9]:
### make review text become lowercase
from pyspark.sql import functions as F
for col in data.columns:
    data = data.withColumn(col, F.lower(F.col(col)))
data.show(5)

+----------+-------+--------------------+--------------------+
|      asin|overall|          reviewText|             summary|
+----------+-------+--------------------+--------------------+
|0700099867|    1.0|installing the ga...|pay to unlock con...|
|0700099867|    4.0|if you like rally...|     good rally game|
|0700099867|    1.0|1st shipment rece...|           wrong key|
|0700099867|    3.0|i got this versio...|awesome game, if ...|
|0700099867|    4.0|i had dirt 2 on x...|              dirt 3|
+----------+-------+--------------------+--------------------+
only showing top 5 rows



In [16]:
data=data.toPandas()

In [18]:
data.head()

,asin,overall,reviewText,summary
0,0700099867,1.0,installing the game was a struggle (because of...,pay to unlock content? i don't think so.
1,0700099867,4.0,if you like rally cars get this game you will ...,good rally game
2,0700099867,1.0,1st shipment received a book instead of the ga...,wrong key
3,0700099867,3.0,"i got this version instead of the ps3 version,...","awesome game, if it did not crash frequently !!"
4,0700099867,4.0,i had dirt 2 on xbox 360 and it was an okay ga...,dirt 3


In [33]:
lst=[]
for line in data:
    lst.append(line)
    

In [ ]:
asin_lst = list(data['asin'])
overall_lst = list(data['overall'])
review_lst = list(data['reviewText'])
summary_lst = list(data['summary'])

In [39]:
import nltk
nltk.download('words')
import nltk
import re
from nltk.corpus import stopwords
from nltk.util import ngrams
import pandas as pd
import nltk

[nltk_data] Downloading package words to /home/hadoop/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


In [49]:
import nltk
nltk.download('stopwords')
nltk.download('words')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /home/hadoop/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to /home/hadoop/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package punkt to /home/hadoop/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [41]:
# get the list of stopwords from nltk
from nltk.corpus import stopwords
eng_stopwords = stopwords.words('english')
eng_stopwords.append('xxxx')

In [45]:
clean_texts = []

In [46]:
words = set(nltk.corpus.words.words())

In [47]:
stop_words = set(stopwords.words('english'))

In [50]:
for text in data["reviewText"]:
    text = " ".join(re.sub("([^0-9A-Za-z \t])|(\w+:\/\/\S+)", "", text).split()) 
    text = text.replace('#', ' ').replace('@', ' ') 

    text = " ".join(w.lower() for w in nltk.wordpunct_tokenize(text) if w.lower() in words) # remove words not in corpus
    text = " ".join(w.lower() for w in nltk.word_tokenize(text) if not w.lower() in stop_words) # remove stop words
    
    clean_texts.append(text)

In [52]:
clean_texts[1]

'like rally get game since huge rally fan party music even game best works'

In [54]:
data['reviewText']=clean_texts

In [55]:
data.head(10)

,asin,overall,reviewText,summary
0,0700099867,1.0,game struggle live championship unlocked game ...,pay to unlock content? i don't think so.
1,0700099867,4.0,like rally get game since huge rally fan party...,good rally game
2,0700099867,1.0,shipment received book instead shipment got fa...,wrong key
3,0700099867,3.0,got version instead version turned mistake con...,"awesome game, if it did not crash frequently !!"
4,0700099867,4.0,dirt game bought new build collection game fun...,dirt 3
5,0700099867,4.0,overall well done racing game good graphics ti...,"good racing game, terrible windows live requir..."
6,0700099867,5.0,dirt thought graphics good dirt addition graph...,a step up from dirt 2 and that is terrific!
7,0700099867,1.0,cant tell piece dog game like everything else ...,crash 3 is correct name aka microsoft
8,0700099867,4.0,initially gave one star constantly recently wh...,a great game ruined by microsoft's account man...
9,0700099867,2.0,still havent figured one everything instructed...,couldn't get this one to work


In [19]:
# remove puctuation
import string
list_punct=list(string.punctuation)

In [24]:
def texts_clean(dataset): 
    lst = []
    for line in dataset:
        lst.append(line)
    
    asin_lst = []
    overall_lst = []
    review_lst = []
    summary_lst = []

    for i in range(len(lst)):
        asin_lst.append(lst[i].split('\t')[0])
        overall_lst.append(lst[i].split('\t')[1])
        review_lst.append(lst[i].split('\t')[2])
        summary_lst.append(lst[i].split('\t')[3])

    data_lst = {'asin': asin_lst, 
                'overall': overall_lst,
                'review': review_lst,
                'summary': summary_lst}
    
    df_lst = pd.DataFrame(data_lst)
   ### Clean the revirews by removing punctuations, numbers, URL, stopwords
    clean_texts = []
    words = set(nltk.corpus.words.words())
    stop_words = set(stopwords.words('english'))

    for text in df_lst["review"]:
    
        text = " ".join(re.sub("([^0-9A-Za-z \t])|(\w+:\/\/\S+)", "", text).split()) 
        text = text.replace('#', ' ').replace('@', ' ') 

        text = " ".join(w.lower() for w in nltk.wordpunct_tokenize(text) if w.lower() in words) # remove words not in corpus
        text = " ".join(w.lower() for w in nltk.word_tokenize(text) if not w.lower() in stop_words) # remove stop words
    
        clean_texts.append(text)
    
    df_lst["clean_texts"] = clean_texts
    df_lst = df_lst.drop(columns="reviews")
    return df_lst




In [25]:
df_clean=texts_clean(data)

IndexError: list index out of range